## Proyecto final
## Curso: BigData
## Programa: Data Science
### Streaming, intregración con Twitter    
### Alonso Nuñez Sanchez

### Importar librerías

In [1]:
import socket
import sys
import requests
import requests_oauthlib
import json
import time

Datos de conexión al API de Twitter.

Previamente fue necesario crear una cuenta de tipo desarrollador en Twitter

In [2]:
ACCESS_TOKEN = '92690221-B4imevK0Bo7hhAlA4CwhVlDq4bUvKTJgaIMM5l3IG'
ACCESS_SECRET = 'g4gA9UfyuRariMz88D6d8D3UWGzNzYQRp5Rc3zeYrjXas'
CONSUMER_KEY = 'anPBf9SWRMF7l2V3SGooxOuEP'
CONSUMER_SECRET = 'zkx9LcWcZYkPFPlJl2T8BjfySy5XKqkXwpXsr1qSRCbqpXkcZA'
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)

usaremos una funcionalidad del API de Twitter para obtener datos en streaming, a partir de ciertas condiciones/filtros

documentación en: https://developer.twitter.com/en/docs/tweets/filter-realtime/api-reference/post-statuses-filter

creamos una función para obtener tweets, la respuesta es un stream de tweets que coincidan con los filtros 

In [3]:
def obtener_tweets():
        	url = 'https://stream.twitter.com/1.1/statuses/filter.json'
        	query_data = [('language', 'es,en'), ('locations', '-85,8,-30,75')] #Centro y Norteamerica
            #query_data = [('language', 'es,en'), ('locations', '-92.5,7.5,-75,17')] #CentroAmerica
            #query_data = [('language', 'es,en'), ('locations', '-85,8,-83,11')] #Costa Rica         
            #query_data = [('language', 'en,es'), ('locations', '-85,8,-30,75'),('track','#')] #Norteamerica
            #query_data = [('language', 'en,es'), ('locations', '-142,-50,-30,75'),('track', '#')] #America            
                       
        	query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
        	response = requests.get(query_url, auth=my_auth, stream=True)
        	print(query_url, response)
        	return response

estructura del archivo json recibido: https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/intro-to-tweet-json          

In [4]:
def enviar_tweets_a_spark(http_resp, tcp_connection):
        for line in http_resp.iter_lines():
            try:
                full_tweet = json.loads(line)
                if full_tweet['entities']['hashtags'] != []: #filtra solo los tweets que contengan hashtags
                    tweet_text = full_tweet['text']                
                    tweet_text_encode = (tweet_text + '\n').encode('utf-8')
                    print(tweet_text)
                    print ("------------------------------------------")
                    
                    tcp_connection.send(tweet_text_encode)                
            except:
                e = sys.exc_info()[1]
                print("Error: %s" % e)

In [ ]:
TCP_IP = "localhost"
TCP_PORT = 9009
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = obtener_tweets()
enviar_tweets_a_spark(resp, conn)

Waiting for TCP connection...
Connected... Starting getting tweets.
https://stream.twitter.com/1.1/statuses/filter.json?language=es,en&locations=-85,8,-30,75 <Response [200]>
#trapmusic
Check out this great gig I've got from #fiverr https://t.co/imhP0Fag0Y
------------------------------------------
Wow oh what a feeling wish my kids could see there mom #RIH
------------------------------------------
